Unificando el scraper

In [9]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image

In [10]:
link_principal='https://www.pagina12.com.ar'
r_web_principal=requests.get(link_principal)

In [11]:
s_web_principal = BeautifulSoup(r_web_principal.text, 'lxml')

In [12]:
secciones_web_principal = s_web_principal.find_all('div', attrs={'class':'p12-dropdown-column'})[0]
link_secciones_web_principal = secciones_web_principal.find_all('a')
#recorremos la lista de tag a
URL_secciones=[]
for i in link_secciones_web_principal:
    if len(link_secciones_web_principal):
        URL_secciones.append(i.get('href'))
URL_secciones


['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/edicion-impresa',
 'https://www.pagina12.com.ar/secciones/universidad-diario',
 'https://www.pagina12.com.ar/secciones/ajedrez',
 'https://www.pagina12.com.ar/secciones/cultura',
 'https://www.pagina12.com.ar/secciones/dialogos',
 'https://www.pagina12.com.ar/secciones/plastica',
 'https://www.pagina12.com.ar/secciones/psicologia',
 'https://www.pagina12.com.ar/secciones/cartas-de-lectores',
 'https://www.pagina12.com.ar/secciones/contratapa',
 'https://www.pagina12.com.ar/secciones/audiovisuales',
 'https://www.pagina12.com.ar/secciones/recordatorios',
 'https://www.pagina12.com.ar/secciones

In [13]:
r_URL_secciones= requests.get(URL_secciones[0])
s_seccion= BeautifulSoup(r_URL_secciones.text, 'lxml')
lista_notas=[]
def obtener_noticias(soup):
    #obtener noticias promocionadas
    noticias_promocionadas=soup.find('div', attrs={'class':'article-item__content'})
    if noticias_promocionadas:
        lista_notas.append(f'{link_principal}{noticias_promocionadas.a.get("href")}')

    #resto de noticias
    resto_noticias=soup.find('section', attrs={'class':'list-content'})
    for i in resto_noticias.find_all('div', attrs={'class':'article-item__header'}):
        if i.a:
            lista_notas.append(f'{link_principal}{i.a.get("href")}')

    return lista_notas


In [14]:
obtener_noticias(s_seccion)

['https://www.pagina12.com.ar/495670-el-abrazo-entre-peron-y-balbin',
 'https://www.pagina12.com.ar/495662-comenzo-el-juicio-por-la-apropiacion-de-victoria-donda',
 'https://www.pagina12.com.ar/495640-cristina-kirchner-recibio-a-evo-morales',
 'https://www.pagina12.com.ar/495637-ataque-al-intendente-de-san-francisco-el-movil-fue-un-tema-d',
 'https://www.pagina12.com.ar/495636-primera-cumbre-de-gobernadoras-y-gobernadores-de-latinoameri',
 'https://www.pagina12.com.ar/495590-cristina-kirchner-recomendo-en-sus-redes-sociales-una-nota-p',
 'https://www.pagina12.com.ar/495588-otro-testigo-declaro-que-sabag-montiel-habria-atentado-contr',
 'https://www.pagina12.com.ar/495551-crece-la-mesa-del-campo-que-alimenta',
 'https://www.pagina12.com.ar/495546-reves-para-jaime-stiuso-no-podra-acceder-a-la-causa-por-el-a',
 'https://www.pagina12.com.ar/495545-reconstruir-la-abogacia-publica',
 'https://www.pagina12.com.ar/495533-atentado-a-cristina-kirchner-agustina-diaz-rompio-el-silenci']

In [30]:
# obtemos informacion de cada nota
URL_nota=lista_notas[0]
r_URL_nota= requests.get(URL_nota)
s_nota=BeautifulSoup(r_URL_nota.text, 'lxml')

def obtener_info(s_nota):
    #creamos un diccionario vacio para guardar informacion de la notas
    info_dic={}

    #extraemos el titulo de la nota
    titulo = s_nota.find('h1').text
    if titulo:
        info_dic['titulo']=titulo
    else:
        info_dic['titulo']=None

    #extraemos la fecha de la nota
    fecha = s_nota.find('time').get_text()
    if fecha:
        info_dic['fecha']=fecha
    else:
        info_dic['fecha']=None

    #extraemos el copete
    copete= s_nota.find('h4').get_text()
    if copete:
        info_dic['copete']=copete
    else:
        info_dic['copete']=None

    #extraemos la volanta
    volanta= s_nota.find('h3').get_text()
    if volanta:
        info_dic['volanta']=volanta
    else:
        info_dic['volanta']=None

    #extraemos el cuerpo de la nota
    cuerpo= s_nota.find('div', attrs={'class':'article-main-content article-text'}).find_all('p')
    cu=[]
    for i in cuerpo:
        cu.append(i.get_text())
    if cu:
        info_dic['cuerpo']=cu
    else:
        info_dic['cuerpo']=None
    
    #extraemos el autor de la nota
    autor= s_nota.find('div', attrs={'class':'author-name'})
    if autor:
        info_dic['autor']=autor.get_text()
    else:
        info_dic['autor']=None
    
    #extraemos la imagen
    #parseamos el tag div que contiene la imagen de la nota
    media = s_nota.find('div', attrs={'class':'article-main-media-image__container'})
    if len(media):
        imagen_src=media.img.get('src')
        try:
            r_img=requests.get(imagen_src)
            if r_img.status_code== 200:
                info_dic['imagen']=r_img.content
            else:
                info_dic['imagen']=None
        except:
            print('No se pudo obtener la imagen')
    else:
        print ('No se encontraron imagenes')

    return  info_dic
#obtiene todos los valores de un diccionario
#print (dic.values())
#obtener un valor de un diccionario, utilizamos la libreria de imagen de python para mostrar la img
#Image(dic.get('imagen'))